In [253]:

# sourcery skip: for-index-underscore
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import json

In [254]:
url = "https://www.tmz.com"
req = requests.get(url)
doc = BeautifulSoup(req.text)

In [255]:
raw_titles = doc.select("header > a > h2")

In [272]:
articles = doc.select("article > a")
articles[0]['href']

'https://www.tmz.com/2022/08/02/fedie-mcclurg-elder-abuse-victim-sexual-assault-conservatorship/'

In [256]:
# Grabs all the headlines, appends period if doesn't end in !
count = 0
headlines = []
valid_punct = ["'", "!", "?"]
for title in enumerate(raw_titles):
    print("- - -")
    headline = raw_titles[count].get_text().strip().replace("\n", " ") + "."
    if headline[-2] in valid_punct:
        headline = headline[:-1]
    else: pass

    print(headline)
    headlines.append(headline)
    count += 1

16it [00:00, 98112.37it/s]

- - -
Vin Scully Dodgers Broadcasting Legend ... Dead At 94.
- - -
Jon Stewart Vet Bill Pressure Works ... GOP Senators Flip, Pass PACT Act.
- - -
Willow Smith Stops Show After Fan Faints in Crowd.
- - -
LeBron James Hits Lakers Facility W/ Sons ... Epic Workout With My Boys!!!
- - -
'Ferris Bueller' Star Edie McClurg Allegedly Elder Abuse Victim ... Caregiver Allegedly Sexually Assaulted.
- - -
Killer Mike ATL's Black Biz Owners in Danger ... I'm Taking a Stand!!!
- - -
Demi Lovato Adds She/Her Pronouns to They/Them ... 'I've Been Feeling More Feminine Recently'
- - -
Kanye West Rips Adidas For 'Yeezy Day' ... I Didn't Approve!!!
- - -
Thomas Markle Tabloid Honcho Files for Restraining Order ... He's Gonna Kill Me!!!
- - -
Deshaun Watson Ban NFL In 'Really Tricky' Appeal Situation ... Andrew Brandt Says.
- - -
Soccer Player Attacks Female Ref Over Yellow Card ... Arrested.
- - -
'Twilight' Star Cam Gigandet Wife Files For Divorce.
- - -
Paraglider Narrowly Cheats Death ... Terrifying 

In [273]:
def get_article_href(article, position):
    articles = doc.select("article > a")
    href = articles[position]['href']
    return href

In [257]:
def get_tags(article, position):
    tags = doc.select("section.tag-cloud > ul > a")
    tag_list = []
    for tag in tags:
        data = json.loads(tag['data-context'])
        if data['pos'] == position:
            tag_list.append(tag.text.strip())
    return tag_list

In [258]:
def break_timestamp(article, position):
    timestamps = doc.select(".article")
        # grab, clean timestamp
    timestamp = timestamps[position].text.split("PT")[-20:]
    timestamp = timestamp[0][-20:]
    timestamp = timestamp.strip()
    # timestamp = datetime.st
    calendar = timestamp[:9]
    clock =  timestamp[-8:].strip()
    
    date_a_frame = pd.DataFrame()
    # date_a_frame["calendar"] = datetime.strptime(calendar, "%b %d %Y %I:%M%p")
    # date_a_frame["clock"] = clock
    year = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").year
    month = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").month
    day = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").day
    hour = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").hour
    minute = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").minute

    datetimes = {
        "calendar": calendar,
        "clock": clock,
        "year": year,
        "month": month,
        "day": day,
        "hour": hour,
        "minute": minute
    }
    meta = []
    meta.append(datetimes)
    return datetimes

In [259]:
break_timestamp(raw_titles, 3)

{'calendar': '8/2/2022 ',
 'clock': '2:57 PM',
 'year': 2022,
 'month': 8,
 'day': 2,
 'hour': 14,
 'minute': 57}

In [260]:
# init dataframe of tags
tags = doc.select("section.tag-cloud > ul > a")

valid_punct = ["'", "!", "?"]

dicts = []

count = 0
for article_position, article in enumerate(raw_titles, start=1):
    headline = raw_titles[count].get_text().strip().replace("\n", " ") + "."
    if headline[-2] in valid_punct:
        headline = headline[:-1]
    tag_list = get_tags(article, count + 1)

    datetimes = break_timestamp(article, count)
    timestamps = doc.select(".article")
    # grab, clean timestamp
    timestamp = timestamps[count].text.split("PT")[-20:]
    timestamp = timestamp[0][-20:]
    timestamp = timestamp.strip()
    
    url = get_article_href(article, count)

    article_dict = {
        "headline": headline,
        "tags": tag_list,
        "timestamp": timestamp,
        "url": url,
        "year": datetimes["year"],
        "month": datetimes["month"],
        "day": datetimes["day"],
        "hour": datetimes["hour"],
        "minute": datetimes["minute"],
    }
    dicts.append(article_dict)
    count += 1
current = pd.DataFrame(dicts)


,headline,tags,timestamp,year,month,day,hour,minute
0,Vin Scully Dodgers Broadcasting Legend ... Dea...,"[Vin Scully, TMZ Sports]",8/2/2022 8:30 PM,2022,8,2,20,30
1,Jon Stewart Vet Bill Pressure Works ... GOP Se...,"[Jon Stewart, Ted Cruz, You Might Want to Reth...",8/2/2022 6:22 PM,2022,8,2,18,22
2,Willow Smith Stops Show After Fan Faints in Cr...,"[Willow Smith, Machine Gun Kelly, Nurse!, Cont...",8/2/2022 6:12 PM,2022,8,2,18,12
3,LeBron James Hits Lakers Facility W/ Sons ... ...,"[LeBron James, Lebron James jr, Bryce James, T...",8/2/2022 2:57 PM,2022,8,2,14,57
4,'Ferris Bueller' Star Edie McClurg Allegedly E...,"[Edie McClurg, Movies, Sex, Nurse!, Exclusive,...",8/2/2022 1:41 PM,2022,8,2,13,41
5,Killer Mike ATL's Black Biz Owners in Danger ....,"[Killer Mike, Hip Hop, Restaurants & Nightclub...",8/2/2022 2:57 PM,2022,8,2,14,57
6,Demi Lovato Adds She/Her Pronouns to They/Them...,"[Demi Lovato, LGBT, Breaking News, Viral, Music]",8/2/2022 1:09 PM,2022,8,2,13,9
7,Kanye West Rips Adidas For 'Yeezy Day' ... I D...,"[Adidas, Kanye West, Money, Hip Hop, Fashion, ...",8/2/2022 1:12 PM,2022,8,2,13,12
8,Thomas Markle Tabloid Honcho Files for Restrai...,"[Meghan Markle, Prince Harry, Family, Exclusiv...",8/2/2022 11:47 AM,2022,8,2,11,47
9,Deshaun Watson Ban NFL In 'Really Tricky' Appe...,"[Deshaun Watson, Cleveland Browns, Houston Tex...",8/2/2022 12:17 PM,2022,8,2,12,17


In [261]:

# This is reading the headlines.csv file and converting it to a dataframe.
existing = pd.read_csv("../headlines.csv")
existing = existing.dropna(axis=1).set_index("Unnamed: 0")
if 'tags' not in existing:
    existing["tags"] = None
existing

,headline,timestamp,year,month,day,hour,minute,tags
Unnamed: 0,,,,,,,,
0,Vin Scully Dodgers Broadcasting Legend ... De...,8/2/2022 8:30 PM,2022,8,2,20,30,None
1,Jon Stewart Vet Bill Pressure Works ... GOP S...,8/2/2022 6:22 PM,2022,8,2,18,22,None
2,Willow Smith Stops Show After Fan Faints in C...,8/2/2022 6:12 PM,2022,8,2,18,12,None
3,Killer Mike ATL's Black Biz Owners in Danger ...,8/2/2022 2:57 PM,2022,8,2,14,57,None
4,'Ferris Bueller' Star Edie McClurg Allegedly ...,8/2/2022 1:41 PM,2022,8,2,13,41,None
...,...,...,...,...,...,...,...,...
154,Taco Bell Employee Allegedly Throws Boiling H...,7/26/2022 6:27 AM,2022,7,26,6,27,None
155,Brittney Griner Sends Message To Wife In Cour...,7/26/2022 6:13 AM,2022,7,26,6,13,None
156,PetSmart Sued Your Groomers Killed My Dog!!!,7/26/2022 1:00 AM,2022,7,26,1,0,None


In [264]:
dfs = [current, existing]
existing = existing[["headline", "tags", "timestamp", "year", "month", "day", "hour", "minute"]]
export = pd.concat([current, existing])

export = export.sort_values(
    by=["year", "month", "day", "hour", "minute"],
    ascending=[False, False, False, False, False],
    ignore_index=True,
)
export = export.drop_duplicates(subset=["timestamp"], keep="first", ignore_index=True)
export

,headline,tags,timestamp,year,month,day,hour,minute
0,Vin Scully Dodgers Broadcasting Legend ... Dea...,"[Vin Scully, TMZ Sports]",8/2/2022 8:30 PM,2022,8,2,20,30
1,Jon Stewart Vet Bill Pressure Works ... GOP Se...,"[Jon Stewart, Ted Cruz, You Might Want to Reth...",8/2/2022 6:22 PM,2022,8,2,18,22
2,Willow Smith Stops Show After Fan Faints in Cr...,"[Willow Smith, Machine Gun Kelly, Nurse!, Cont...",8/2/2022 6:12 PM,2022,8,2,18,12
3,LeBron James Hits Lakers Facility W/ Sons ... ...,"[LeBron James, Lebron James jr, Bryce James, T...",8/2/2022 2:57 PM,2022,8,2,14,57
4,'Ferris Bueller' Star Edie McClurg Allegedly E...,"[Edie McClurg, Movies, Sex, Nurse!, Exclusive,...",8/2/2022 1:41 PM,2022,8,2,13,41
...,...,...,...,...,...,...,...,...
154,Taco Bell Employee Allegedly Throws Boiling H...,None,7/26/2022 6:27 AM,2022,7,26,6,27
155,Brittney Griner Sends Message To Wife In Cour...,None,7/26/2022 6:13 AM,2022,7,26,6,13
156,PetSmart Sued Your Groomers Killed My Dog!!!,None,7/26/2022 1:00 AM,2022,7,26,1,0
157,'RHOA' STAR SHEREE WHITFIELD Dating Martell H...,None,7/26/2022 12:50 AM,2022,7,26,0,50


In [252]:
# Writing the dataframe to a csv file.
export.to_csv("headlines.csv")
export.to_json("headlines.json")
